In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train_data = pd.read_csv('/kaggle/input/titanic/train.csv')

In [ ]:
def pandas_profile(df):
    print(train_data.describe())
    print(train_data.info())
    from pandas_profiling import ProfileReport
    prof = ProfileReport(df)
    prof.to_file(output_file='report.html')

In [ ]:
 pandas_profile(train_data)

In [ ]:
from IPython.display import HTML
report = './report.html'
HTML(filename=report)

In [ ]:
train_data_cpy = train_data.drop(['Cabin', 'PassengerId'], axis=1)

In [ ]:
train_data_cpy['Age'].fillna((train_data_cpy['Age'].mean()), inplace=True)

In [ ]:
train_data_cpy.info()

In [ ]:
from sklearn_pandas import CategoricalImputer
imputer = CategoricalImputer()
train_data_cpy['Embarked'] = imputer.fit_transform(train_data_cpy['Embarked'])

In [ ]:
train_data_cpy.info()

In [ ]:
lst = list(train_data_cpy['Embarked'])
lst

In [ ]:
def load_dataset(data):
    # load the dataset as a pandas DataFrame
    # retrieve numpy array
    from sklearn_pandas import CategoricalImputer
    y = data['Survived']
    y_train = np.asarray(y).astype('float32')
    data = data.drop(['Survived'], axis=1)
    categorical_data = data.select_dtypes(exclude=[np.number])
    converted_categorical_data = dummyEncode(data)
    numerical_data = data.select_dtypes(include=[np.number])
    converted_numerical_data = normalize_numeric_data(numerical_data)
    df_col = pd.concat([converted_categorical_data,converted_numerical_data], axis=1)
    X = df_col.to_numpy()
    return X, y_train

def dummyEncode(df):
    from sklearn_pandas import CategoricalImputer
    columnsToEncode = list(df.select_dtypes(include=['category','object']))
    le = LabelEncoder()
    for feature in columnsToEncode:
        try:
            df[feature] = le.fit_transform(df[feature])
        except Exception as e:
            print(e)
            print('Error encoding '+feature)
    return df

# prepare input data
def normalize_numeric_data(data):
    from sklearn import preprocessing
    x = data.values #returns a numpy array
    min_max_scaler = preprocessing.MinMaxScaler()
    x_scaled = min_max_scaler.fit_transform(x)
    df = pd.DataFrame(x_scaled)
    return df



In [ ]:
X, y = load_dataset(train_data_cpy)

In [ ]:
X

In [ ]:
y

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)

In [ ]:
from keras import models
from keras import layers

def build_model():
    model = models.Sequential() 
    model.add(layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],))) 
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))
    model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc']) 
    history = model.fit(X_train, y_train, validation_data=(X_test, y_test),epochs=20, batch_size=1, verbose=1)
    return history


In [ ]:
history = build_model()


In [ ]:
history_dict = history.history
history_dict.keys()

In [ ]:
import matplotlib.pyplot as plt
history_dict = history.history
loss_values = history_dict['loss'] 
acc = history_dict['acc'] 
val_loss_values = history_dict['val_loss']
epochs = range(1, len(acc) + 1)
plt.plot(epochs, loss_values, 'bo', label='Training loss') 
plt.plot(epochs, val_loss_values, 'b', label='Validation loss')
plt.title('Training and validation loss') 
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
plt.clf() 
acc_values = history_dict['acc'] 
val_acc_values = history_dict['val_acc']
plt.plot(epochs, acc, 'bo', label='Training acc') 
plt.plot(epochs, val_acc_values, 'b', label='Validation acc') 
plt.title('Training and validation accuracy') 
plt.xlabel('Epochs')
plt.ylabel('Loss') 
plt.legend()
plt.show()